Initial run for Isabel's Data
=================

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
basedir = '../'
sys.path.append(basedir)

import numpy as np
import pandas as pd
import pylab as plt
from IPython.display import display
import math

from lda_for_fragments import Ms2Lda

<h2>Analysis</h2>

<h3>a. Run LDA</h3>

In [2]:
fragment_filename = 'isabelpos_fragments.csv'
neutral_loss_filename = 'isabelpos_losses.csv'
mzdiff_filename = None
ms1_filename = 'isabelpos_ms1.csv'
ms2_filename = 'isabelpos_ms2.csv'
ms2lda = Ms2Lda.lcms_data_from_R(fragment_filename, neutral_loss_filename, mzdiff_filename, 
                             ms1_filename, ms2_filename)

Loading input files
Data shape (435, 1108)


In [2]:
### all the parameters you need to specify to run LDA ###

n_topics = 100
n_samples = 1000
n_burn = 0 # if 0 then we only use the last sample
n_thin = 1 # every n-th sample to use for averaging after burn-in. Ignored if n_burn = 0
alpha = 50.0/n_topics # hyper-parameter for document-topic distributions
beta = 0.1 # hyper-parameter for topic-word distributions

# ms2lda.run_lda(n_topics, n_samples, n_burn, n_thin, alpha, beta)

<h3>b. Annotations</h3>

In [3]:
# sirius_platform = 'orbitrap'
# ms2lda.annotate_with_sirius(sirius_platform, mode='pos', max_ms1=9999) # mode is either 'pos' or 'neg'

In [21]:
ms2lda = Ms2Lda.resume_from('isabel.project')
proton_mass = 1.00727645199076

# peaklist_filename = 'peaklist_POS.csv'
peaklist_filename = 'sanitized_ideom_output.csv'
peaklist = pd.read_csv(peaklist_filename)
annot = peaklist[['mz', 'rt', 'int', 'metabolite','ttest: AN6306']]
# annot = annot[pd.notnull(annot['Metabolite'])] # drop NaN metabolites


label_len = 20
mass_tol = 10
rt_tol = 120

# for each row in the IDIOM annotation results 
total_match = 0
for row in annot.itertuples():
    
    # get mz, rt, intensity and the putative metabolite identity
    mz = row[1] + proton_mass
    rt = row[2]
    intensity = row[3]
    label = row[4]
    fold_change = row[5]
    print fold_change
    
    # truncate long metabolite names
#     label = (label[:label_len] + '..') if len(label) > label_len else label    
    print "Processing " + str((mz, rt, intensity)) + " " + str(label)

    # find matching MS1 within mass_tol and rt_tol
    interval = mz*mass_tol*1e-6
    mass_start, mass_end = (mz-interval, mz+interval)
    rt_start, rt_end = (rt-rt_tol, rt+rt_tol)
    
    match_mass = (ms2lda.ms1.mz>mass_start) & (ms2lda.ms1.mz<mass_end)
    match_rt = (ms2lda.ms1.rt>rt_start) & (ms2lda.ms1.rt<rt_end)
    match = match_mass & match_rt
    
    res = ms2lda.ms1[match]
    if len(res) == 1:

        # if only 1 match, immediately annotate
        print "Found 1 hit"
        idx = res.index[0]
        ms2lda.ms1.loc[[idx], 'annotation'] = label
        ms2lda.ms1.loc[[idx], 'fold_change'] = fold_change
        display(res)
        total_match += 1

    elif len(res) > 0:

        print "Found %d hits" % len(res)        
        
        # if multiple matches, find the closest in RT   
        closest = None
        min_dist = sys.maxint
        for match_res in res.itertuples():
            match_rt = match_res[4]
            match_mz = match_res[5]
            dist = math.sqrt((match_rt-rt)**2 + (match_mz-mz)**2)
            if dist < min_dist:
                min_dist = dist
                closest = match_res

        idx = closest[0]
        ms2lda.ms1.loc[[idx], 'annotation'] = label
        ms2lda.ms1.loc[[idx], 'fold_change'] = fold_change
        display(res)
        total_match += 1        
        
    print
    
print "total_match=" + str(total_match) + "/" + str(annot.shape[0])

Project loaded from isabel.project time taken = 4.82553195953
 - input_filenames = 
	isabelpos_fragments.csv
	isabelpos_losses.csv
	isabelpos_ms1.csv
	isabelpos_ms2.csv
 - df.shape = (435, 1108)
 - K = 100
 - alpha = 0.5
 - beta = 0.1
 - number of samples stored = 1
 - last_saved_timestamp = Thu Oct  8 18:19:16 2015
 - message = Initial run on Isabel data by Joe
0.850333225
Processing (71.085536421990767, 232.57195980000003, 34996) nan

0.957232192
Processing (74.023649131990766, 695.99287670000012, 3532) Iminoglycine

0.36484501
Processing (74.060028581990764, 595.91434649999997, 48715) Aminoacetone
Found 2 hits


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation
3888,3888,0,1,508.189,74.060018,1482000.625,1,20,0,C3H7NO
5502,5502,0,1,615.361,74.060035,4867594.000,1,0,0,Aminoacetone



0.596061906
Processing (76.039059521990765, 1356.103124, 29573) nan

0.510957158
Processing (76.039295691990759, 822.14852840000003, 245819) Glycine

0.142283491
Processing (76.075680391990758, 648.14350860000002, 57397) (R)-1-Aminopropan-2-ol

0.537264769
Processing (77.008135031990761, 1380.0902100000001, 22659) nan

0.693673882
Processing (77.023065081990765, 410.51318800000001, 757550) nan

0.903409373
Processing (77.649609521990769, 257.633667, 2485) nan

0.389405063
Processing (78.099681331990766, 256.86133239999998, 6149) nan

0.315635686
Processing (78.211064611990764, 253.77637510000002, 14225) nan

0.809781998
Processing (78.984402421990765, 1287.2287550000001, 8819) nan

0.028865142
Processing (79.021203161990769, 408.629795, 130333) nan

0.97263044
Processing (79.021206391990759, 5.9989650879999994, 109253) nan

0.908654414
Processing (80.933275081990757, 717.23124689999997, 42302) Br-

0.079473465
Processing (80.947727751990755, 721.29287470000008, 16973) nan

0.454293783

,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
3606,3606,0,1,536.499,86.096339,15411655,1,0,0,C5H11N,NaN
3607,3607,0,1,521.822,86.096340,11351523,1,77,0,C5H11N,NaN



0.082817609
Processing (87.043988721990758, 655.28750100000002, 4522) 2-Butenoate

0.638787959
Processing (87.044030321990761, 794.88474659999997, 56969) Diacetyl

0.946841127
Processing (87.091696071990768, 1231.804695, 166322) nan

0.137328509
Processing (87.10241521199076, 644.7585004, 3004) nan

0.832594646
Processing (88.039292101990767, 801.01649860000009, 15314) 2-Aminoacrylate

0.432065184
Processing (89.023254561990768, 462.82610319999998, 2810079) Pyruvate

0.678287127
Processing (89.023315021990769, 784.43687179999995, 352474) 3-Oxopropanoate

0.609083305
Processing (90.018562751990757, 729.32312390000004, 13831) Oxamate

0.283339496
Processing (90.054894181990761, 791.51574579999999, 10722286) L-Alanine

0.585873229
Processing (90.054958671990761, 1336.0123390000001, 391083) Sarcosine

0.745908589
Processing (90.054959461990762, 1162.7030600000001, 185944) Sarcosine

0.562955121
Processing (90.054974261990765, 1243.8015809999999, 450100) D-Alanine

0.835128347
Processing (

...


nan
Processing (834.62329785199074, 241.6501083, 14244) PS(17:0/22:0)

nan
Processing (836.54240585199068, 214.16510449999998, 30969) PS(40:6)

nan
Processing (838.55784855199067, 213.59152090000003, 21442) PS(40:5)

nan
Processing (838.62959105199081, 237.905688, 313188) PC(40:4)

nan
Processing (840.57403615199087, 212.33837450000001, 40241) PS(40:4)

nan
Processing (842.58937335199073, 212.09633259999998, 46975) PS(40:3)

nan
Processing (844.60523695199072, 212.45361589999999, 38908) PS(40:2)

nan
Processing (851.6690241519907, 912.05187609999996, 5946) nan

nan
Processing (858.59831385199072, 237.50433380000001, 145153) PC(42:8)

nan
Processing (868.57211105199076, 240.94289320000001, 19908) nan

nan
Processing (877.26287545199079, 200.86426530000003, 1773) nan

nan
Processing (880.58320985199066, 236.70545959999998, 163245) PC(44:11)
Found 1 hit


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
718,718,0,1,205.485,880.590255,864663.9375,1,0,0,"[PI (18:0/18:0)] 1,2..",NaN



nan
Processing (880.59001535199081, 219.55060449999999, 94209) [PI (18:0/18:0)] 1,2-di-(9Z-octadecenoyl)-sn-glycero-3-phospho-(1'-myo-inositol)(ammonium salt)
Found 1 hit


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
718,718,0,1,205.485,880.590255,864663.9375,1,0,0,PC(44:11),NaN



nan
Processing (882.59855175199061, 236.7771889, 87839) PC(44:10)

nan
Processing (892.57175865199076, 240.29648950000001, 10169) nan

nan
Processing (975.38491285199075, 587.93751529999997, 128634) nan

total_match=97/1154


<h3>c. Save Project</h3>

In [7]:
# leave the message parameter out if nothing to say
# ms2lda.save_project('isabel.project', message="Initial run on Isabel data by Joe")

Project saved to isabel.project time taken = 2.27142715454


<hr/>

<h2>Results</h2>

In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = '../'
sys.path.append(basedir)

from lda_for_fragments import Ms2Lda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
ms2lda = Ms2Lda.resume_from('isabel.project')

Project loaded from isabel.project time taken = 5.01552891731
 - input_filenames = 
	isabelpos_fragments.csv
	isabelpos_losses.csv
	isabelpos_ms1.csv
	isabelpos_ms2.csv
 - df.shape = (435, 1108)
 - K = 100
 - alpha = 0.5
 - beta = 0.1
 - number of samples stored = 1
 - last_saved_timestamp = Thu Oct  8 18:19:16 2015
 - message = Initial run on Isabel data by Joe


In [22]:
# Thresholding the doc_topic and topic_word matrices
ms2lda.do_thresholding(th_doc_topic=0.05, th_topic_word=0.01)

In [23]:
# ms2lda.print_topic_words()

In [26]:
special_nodes = []
for i in ms2lda.ms1.index:
    fold_change = ms2lda.ms1.loc[i,'fold_change']
    if fold_change <= 0.3:
        print i
        special_nodes.append(("doc_{}".format(i),'#CC0000'))
print special_nodes

1730
2998
3546
4051
4139
5489
6525
6580
[('doc_1730', '#CC0000'), ('doc_2998', '#CC0000'), ('doc_3546', '#CC0000'), ('doc_4051', '#CC0000'), ('doc_4139', '#CC0000'), ('doc_5489', '#CC0000'), ('doc_6525', '#CC0000'), ('doc_6580', '#CC0000')]


In [27]:
interactive=True
consistency=0.0
# special_nodes = None
ms2lda.plot_lda_fragments(consistency=consistency, interactive=interactive, to_highlight=special_nodes)

Ranking topics ...
 - topic 0 h-index = 2
 - topic 1 h-index = 5
 - topic 2 h-index = 1
 - topic 3 h-index = 4
 - topic 4 h-index = 2
 - topic 5 h-index = 2
 - topic 6 h-index = 3
 - topic 7 h-index = 3
 - topic 8 h-index = 1
 - topic 9 h-index = 2
 - topic 10 h-index = 2
 - topic 11 h-index = 2
 - topic 12 h-index = 1
 - topic 13 h-index = 4
 - topic 14 h-index = 7
 - topic 15 h-index = 1
 - topic 16 h-index = 2
 - topic 17 h-index = 3
 - topic 18 h-index = 3
 - topic 19 h-index = 5
 - topic 20 h-index = 2
 - topic 21 h-index = 4
 - topic 22 h-index = 3
 - topic 23 h-index = 2
 - topic 24 h-index = 2
 - topic 25 h-index = 2
 - topic 26 h-index = 2
 - topic 27 h-index = 2
 - topic 28 h-index = 2
 - topic 29 h-index = 7
 - topic 30 h-index = 2
 - topic 31 h-index = 4
 - topic 32 h-index = 5
 - topic 33 h-index = 3
 - topic 34 h-index = 1
 - topic 35 h-index = 2
 - topic 36 h-index = 2
 - topic 37 h-index = 3
 - topic 38 h-index = 3
 - topic 39 h-index = 7
 - topic 40 h-index = 2
 - topi

127.0.0.1 - - [12/Oct/2015 20:05:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:18] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:18] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:18] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:18] "GET /images/graph_example.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:18] "GET /images/default_logo.png HTTP/1.1" 200 -


Serving dynamic json file -- threshold = 0
to_highlight = [('doc_1730', '#CC0000'), ('doc_2998', '#CC0000'), ('doc_3546', '#CC0000'), ('doc_4051', '#CC0000'), ('doc_4139', '#CC0000'), ('doc_5489', '#CC0000'), ('doc_6525', '#CC0000'), ('doc_6580', '#CC0000')]
43, topic_28 degree=5 added

127.0.0.1 - - [12/Oct/2015 20:05:23] "GET /graph.html?degree=0&visID=ldavis_el953344843238803058776664 HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 20:05:23] "GET /graph.json HTTP/1.1" 200 -



47, topic_24 degree=3 added
48, topic_25 degree=4 added
49, topic_22 degree=7 added
50, topic_23 degree=2 added
52, topic_21 degree=16 added
59, topic_91 degree=15 added
60, topic_90 degree=24 added
61, topic_97 degree=4 added
62, topic_96 degree=10 added
63, topic_95 degree=6 added
64, topic_94 degree=5 added
66, topic_99 degree=3 added
67, topic_98 degree=5 added
70, topic_0 degree=3 added
129, topic_39 degree=28 added
130, topic_68 degree=5 added
131, topic_69 degree=5 added
133, topic_62 degree=1 added
134, topic_63 degree=34 added
135, topic_60 degree=16 added
136, topic_61 degree=3 added
137, topic_66 degree=3 added
138, topic_67 degree=6 added
139, topic_64 degree=9 added
140, topic_65 degree=15 added
181, topic_7 degree=13 added
182, topic_6 degree=6 added
183, topic_5 degree=3 added
184, topic_4 degree=4 added
185, topic_3 degree=5 added
186, topic_2 degree=2 added
187, topic_1 degree=9 added
190, topic_9 degree=3 added
191, topic_8 degree=1 added
197, topic_70 degree=6 added